In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
# Read the training and validation CSV files
train_df = pd.read_csv("examples/data/train_hc_dataset_small_20240915_044230.csv")
val_df = pd.read_csv("examples/data/val_hc_dataset_small_20240915_044230.csv")


# Preprocessing function
def preprocess_data(df):
    # Parse 'prob_diff_values' into list of floats
    df["prob_diff_sequence"] = df["prob_diff_values"].apply(ast.literal_eval)
    df["cosine_sequence"] = df["cosine_values"].apply(ast.literal_eval)

    # Encode 'verdict' into numerical labels
    le = LabelEncoder()
    df["verdict_label"] = le.fit_transform(df["verdict"])

    # Pad sequences to the maximum length
    max_prob_diff_len = df["prob_diff_sequence"].apply(len).max()
    max_cosine_len = df["cosine_sequence"].apply(len).max()

    def pad_sequence(seq, max_len):
        return seq + [0] * (max_len - len(seq))

    prob_diff_sequence_padded = np.array(
        df["prob_diff_sequence"].apply(lambda x: pad_sequence(x, max_prob_diff_len))
    )
    prob_diff_sequence_padded_df = pd.DataFrame(
        prob_diff_sequence_padded.tolist(),
        columns=[f"prob_diff_sequence_padded_{i}" for i in range(max_prob_diff_len)],
    )

    cosine_sequence_padded = np.array(
        df["cosine_sequence"].apply(lambda x: pad_sequence(x, max_cosine_len))
    )

    cosine_sequence_padded_df = pd.DataFrame(
        cosine_sequence_padded.tolist(),
        columns=[f"cosine_sequence_padded_{i}" for i in range(max_cosine_len)],
    )

    df = pd.concat([df, prob_diff_sequence_padded_df, cosine_sequence_padded_df], axis=1)

    # Compute statistical features
    df["prob_diff_mean"] = df["prob_diff_sequence"].apply(np.mean)
    df["prob_diff_std"] = df["prob_diff_sequence"].apply(np.std)
    df["prob_diff_max"] = df["prob_diff_sequence"].apply(np.max)
    df["prob_diff_min"] = df["prob_diff_sequence"].apply(np.min)
    df["prob_diff_median"] = df["prob_diff_sequence"].apply(np.median)
    df["prob_diff_q25"] = df["prob_diff_sequence"].apply(lambda x: np.percentile(x, 25))
    df["prob_diff_q75"] = df["prob_diff_sequence"].apply(lambda x: np.percentile(x, 75))

    df["cosine_mean"] = df["cosine_sequence"].apply(np.mean)
    df["cosine_std"] = df["cosine_sequence"].apply(np.std)
    df["cosine_max"] = df["cosine_sequence"].apply(np.max)
    df["cosine_min"] = df["cosine_sequence"].apply(np.min)
    df["cosine_median"] = df["cosine_sequence"].apply(np.median)
    df["cosine_q25"] = df["cosine_sequence"].apply(lambda x: np.percentile(x, 25))
    df["cosine_q75"] = df["cosine_sequence"].apply(lambda x: np.percentile(x, 75))

    return df


# Preprocess training and validation data
train_df = preprocess_data(train_df)
val_df = preprocess_data(val_df)

# Define feature columns
prob_diff_features = [
    "prob_diff_mean",
    "prob_diff_std",
    "prob_diff_max",
    "prob_diff_min",
    "prob_diff_median",
    "prob_diff_q25",
    "prob_diff_q75",
]

cosine_features = [
    "cosine_mean",
    "cosine_std",
    "cosine_max",
    "cosine_min",
    "cosine_median",
    "cosine_q25",
    "cosine_q75",
]

prob_diff_sequence_cols = [c for c in train_df.columns if "prob_diff_sequence_padded" in c]
cosine_sequence_cols = [c for c in train_df.columns if "cosine_sequence_padded" in c]

In [3]:
# Define datasets
class HallucinationDataset(Dataset):
    def __init__(self, df, X_columns, y_column):
        self.sequences = df[X_columns].values
        self.labels = df[y_column].values

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = torch.tensor(self.sequences[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return sequence, label

    def sample(self, n):
        idx = np.random.choice(len(self), n, replace=False)
        return self.__getitem__(idx)


# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for sequences, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for sequences, labels in val_loader:
                outputs = model(sequences)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(
            f"Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}, Val Accuracy: {100 * correct/total}%"
        )

In [4]:
# Prepare datasets and dataloaders
batch_size = 128

train_dataset = HallucinationDataset(train_df, prob_diff_sequence_cols, "verdict_label")
val_dataset = HallucinationDataset(val_df, prob_diff_sequence_cols, "verdict_label")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

stats_train_dataset = HallucinationDataset(train_df, prob_diff_features, "verdict_label")
stats_val_dataset = HallucinationDataset(val_df, prob_diff_features, "verdict_label")

stats_train_loader = DataLoader(stats_train_dataset, batch_size=batch_size, shuffle=True)
stats_val_loader = DataLoader(stats_val_dataset, batch_size=batch_size, shuffle=False)

In [5]:
class LinearClassifier(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(LinearClassifier, self).__init__()

        self.weights = nn.Linear(input_shape, output_shape)

    def forward(self, x):
        return self.weights(x)


X_sample, y_sample = stats_train_dataset.sample(1)

# Instantiate the model
model = LinearClassifier(input_shape=X_sample.shape[-1], output_shape=2)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, stats_train_loader, stats_val_loader, criterion, optimizer, epochs=20)

LinearClassifier(
  (weights): Linear(in_features=7, out_features=2, bias=True)
)


/Users/jessicarumbelow/PIZZA/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/20, Train Loss: 0.6687743466811119, Val Loss: 0.6492811921817153, Val Accuracy: 66.14090431125132%
Epoch 2/20, Train Loss: 0.6385472736645155, Val Loss: 0.6337828075707849, Val Accuracy: 67.09896015889707%
Epoch 3/20, Train Loss: 0.6273689262345113, Val Loss: 0.6273951623866807, Val Accuracy: 66.94707325622151%
Epoch 4/20, Train Loss: 0.6216344142676423, Val Loss: 0.6242937988309718, Val Accuracy: 67.1223273746933%
Epoch 5/20, Train Loss: 0.6184488386555291, Val Loss: 0.6221186104995101, Val Accuracy: 67.4261011800444%
Epoch 6/20, Train Loss: 0.615961301736054, Val Loss: 0.6203376081452441, Val Accuracy: 67.44946839584064%
Epoch 7/20, Train Loss: 0.6142521393145615, Val Loss: 0.618952736925723, Val Accuracy: 67.58967169061806%
Epoch 8/20, Train Loss: 0.6122803048514501, Val Loss: 0.6178583778552155, Val Accuracy: 67.65977333800677%
Epoch 9/20, Train Loss: 0.6108433632379949, Val Loss: 0.6180352266155072, Val Accuracy: 67.32094870896132%
Epoch 10/20, Train Loss: 0.60977580798030

In [6]:
class LinearClassifier(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(LinearClassifier, self).__init__()

        self.weights = nn.Linear(input_shape, output_shape)

    def forward(self, x):
        return self.weights(x)


X_sample, y_sample = train_dataset.sample(1)

# Instantiate the model
model = LinearClassifier(input_shape=X_sample.shape[-1], output_shape=2)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

LinearClassifier(
  (weights): Linear(in_features=296, out_features=2, bias=True)
)
Epoch 1/20, Train Loss: 0.6572112893853577, Val Loss: 0.6484712313360242, Val Accuracy: 63.290103984110296%
Epoch 2/20, Train Loss: 0.6435949336817336, Val Loss: 0.6453771675700573, Val Accuracy: 63.07979904194415%
Epoch 3/20, Train Loss: 0.6402308342282864, Val Loss: 0.6442113024974937, Val Accuracy: 63.00969739455544%
Epoch 4/20, Train Loss: 0.6381710660304123, Val Loss: 0.644451797898136, Val Accuracy: 61.28052342563384%
Epoch 5/20, Train Loss: 0.6370994323312981, Val Loss: 0.6435991676885691, Val Accuracy: 62.191844841687114%
Epoch 6/20, Train Loss: 0.6360173678193481, Val Loss: 0.6427220288497298, Val Accuracy: 63.02138100245356%
Epoch 7/20, Train Loss: 0.6351033400568328, Val Loss: 0.6425115328226516, Val Accuracy: 63.00969739455544%
Epoch 8/20, Train Loss: 0.634503760051318, Val Loss: 0.6422100983448883, Val Accuracy: 62.78770884449118%
Epoch 9/20, Train Loss: 0.6340355765666061, Val Loss: 0.6441

In [7]:
class SimpleNN(nn.Module):
    def __init__(self, input_shape, output_shape, breadth, depth, interpolate=True):
        super(SimpleNN, self).__init__()

        self.layers = []

        self.layers.append(nn.Linear(input_shape, breadth))
        self.layers.append(nn.ReLU())

        if interpolate:
            change_rate = int((breadth - output_shape) / max(1, depth))
            layer_sizes = [breadth - change_rate * i for i in range(depth)]

            for l in layer_sizes[:-1]:
                self.layers.append(nn.Linear(l, l - change_rate))
                self.layers.append(nn.ReLU())

            self.layers.append(nn.Linear(layer_sizes[-1], output_shape))

        else:
            for _ in range(depth - 1):
                self.layers.append(nn.Linear(breadth, breadth))
                self.layers.append(nn.ReLU())
            self.layers.append(nn.Linear(breadth, output_shape))

        self.layers = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.layers(x)


X_sample, y_sample = train_dataset.sample(1)

# Instantiate the model
model = SimpleNN(input_shape=X_sample.shape[-1], output_shape=2, breadth=64, depth=2)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

SimpleNN(
  (layers): Sequential(
    (0): Linear(in_features=296, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=33, bias=True)
    (3): ReLU()
    (4): Linear(in_features=33, out_features=2, bias=True)
  )
)
Epoch 1/20, Train Loss: 0.6092315034804938, Val Loss: 0.6036146369887821, Val Accuracy: 68.5360439303657%
Epoch 2/20, Train Loss: 0.5789172593104481, Val Loss: 0.5842429308304146, Val Accuracy: 70.19511625189858%
Epoch 3/20, Train Loss: 0.5663356641587269, Val Loss: 0.5818815427039986, Val Accuracy: 70.0782801729174%
Epoch 4/20, Train Loss: 0.5611574817368913, Val Loss: 0.5735514570972813, Val Accuracy: 70.97791798107255%
Epoch 5/20, Train Loss: 0.5535842984786873, Val Loss: 0.5735278345310866, Val Accuracy: 70.77929664680454%
Epoch 6/20, Train Loss: 0.5483545504926101, Val Loss: 0.5691738064164547, Val Accuracy: 70.91949994158196%
Epoch 7/20, Train Loss: 0.5448523640888443, Val Loss: 0.5755075692240872, Val Accuracy: 70.21848346769482%
E

In [8]:
class SimpleConvNet(nn.Module):
    def __init__(self, input_shape, output_shape, breadth, depth):
        super(SimpleConvNet, self).__init__()

        layers = []
        in_channels = input_shape[0]
        ks = 3
        num_filters = breadth // (ks) ** 2

        # Add convolutional layers that maintain the input dimension
        for _ in range(depth):
            layers.append(nn.Conv1d(in_channels, num_filters, kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            in_channels = num_filters

        self.conv_layers = nn.Sequential(*layers)

        # Calculate the size of the flattened output
        conv_output_shape = input_shape[1] * input_shape[2] * num_filters

        # Fully connected layer for classification
        self.fc = nn.Linear(conv_output_shape, output_shape)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)
        return x


X_sample, y_sample = train_dataset.sample(1)
input_shape = X_sample.unsqueeze(1).shape
# Instantiate the model
model = SimpleConvNet(input_shape=input_shape, output_shape=2, breadth=64, depth=2)
print(model)

# Rest of the setup remains the same
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

SimpleConvNet(
  (conv_layers): Sequential(
    (0): Conv1d(1, 7, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(7, 7, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
  )
  (fc): Linear(in_features=2072, out_features=2, bias=True)
)
Epoch 1/20, Train Loss: 0.6954296228711697, Val Loss: 0.6936510943654758, Val Accuracy: 49.66701717490361%
Epoch 2/20, Train Loss: 0.690574354368218, Val Loss: 0.6647087691435173, Val Accuracy: 61.595980838883044%
Epoch 3/20, Train Loss: 0.6384924934145718, Val Loss: 0.623130173825506, Val Accuracy: 66.7484519219535%
Epoch 4/20, Train Loss: 0.6160503472381396, Val Loss: 0.6093393501950733, Val Accuracy: 68.03364879074658%
Epoch 5/20, Train Loss: 0.6026682851140591, Val Loss: 0.6176767847431239, Val Accuracy: 65.4515714452623%
Epoch 6/20, Train Loss: 0.5929928994997377, Val Loss: 0.5993154189035074, Val Accuracy: 67.62472251431242%
Epoch 7/20, Train Loss: 0.5885375080702131, Val Loss: 0.5972273698938426, Val Accurac

In [9]:
class SimpleRNN(nn.Module):
    def __init__(self, input_shape, output_shape, breadth, depth):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(
            input_size=input_shape,
            hidden_size=breadth,
            num_layers=depth,
            batch_first=True,
            bidirectional=True,
        )
        self.fc = nn.Linear(breadth, output_shape)

    def forward(self, x):
        x = x.unsqueeze(1)
        _, h_n = self.rnn(x)
        out = self.fc(h_n[-1])
        return out


# Sample a batch from the dataset
X_sample, y_sample = train_dataset.sample(1)
input_shape = X_sample.shape[-1]  # Extract the number of features

# Instantiate the model
model = SimpleRNN(input_shape=input_shape, output_shape=2, breadth=64, depth=2)
print(model)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

SimpleRNN(
  (rnn): RNN(296, 64, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)
Epoch 1/20, Train Loss: 0.6395679517876949, Val Loss: 0.6163862142989884, Val Accuracy: 67.82334384858044%
Epoch 2/20, Train Loss: 0.6051266115366645, Val Loss: 0.6087914416149481, Val Accuracy: 67.48451921953499%
Epoch 3/20, Train Loss: 0.5881137239063247, Val Loss: 0.5899924920566046, Val Accuracy: 69.03843906998482%
Epoch 4/20, Train Loss: 0.5845461357304979, Val Loss: 0.5895802405343127, Val Accuracy: 69.54083420960393%
Epoch 5/20, Train Loss: 0.5731739075654566, Val Loss: 0.5823497325181961, Val Accuracy: 69.62261946489076%
Epoch 6/20, Train Loss: 0.5731813173437323, Val Loss: 0.6078525699341475, Val Accuracy: 66.72508470615726%
Epoch 7/20, Train Loss: 0.5693853175179641, Val Loss: 0.5830367970377651, Val Accuracy: 69.90302605444562%
Epoch 8/20, Train Loss: 0.5650829755952942, Val Loss: 0.5819652878971242, Val Accuracy: 70.52225727304592%